In [1]:
#1
import numpy as np

#data = np.load("C:/Users/calvi/Desktop/School/ANSC480/DataCode/results_dataset/2019-11-27--10_24_59/000200/behaviour_25.npy", allow_pickle=True)
data_november_29 = []
for i in range(200, 315):
    data = np.load("C:/Users/calvi/Desktop/School/ANSC480/DataCode/results_dataset/2019-11-27--10_24_59/000" + str(i) + "/behaviour_15.npy", allow_pickle=True)
    data_november_29.append(data)

In [2]:
#2
import cv2
from datetime import datetime

def load_frames(video_index):
    video = cv2.VideoCapture("C:/Users/calvi/Desktop/School/ANSC480/DataCode/PIGS291119/000" + str(video_index) + "/color.mp4")

    frames = []


    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        frames.append(frame)

    video.release()
    return frames

def load_times(video_index):
    times = []
    with open("C:/Users/calvi/Desktop/School/ANSC480/DataCode/PIGS291119/000" + str(video_index) + "/times.txt", "r") as file:
        for line in file:
            times.append(get_time(line.strip()))
    return times

def is_eating(i):
    if i == 4:
        return True
    else:
        return False

def at_feeder(i):
    return i == 1

def display_frame(frame_data, frame):
    frame_copy = frame.copy()
    i = 9
    x1 = int(frame_data[1])
    y1 = int(frame_data[2])
    x2 = int(frame_data[3])
    y2 = int(frame_data[4])
    #print(data[i][1][0])
    cv2.rectangle(frame_copy, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)
    frame_copy_rgb = cv2.cvtColor(frame_copy, cv2.COLOR_BGR2RGB)
    cv2.imshow('Frame', frame_copy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def get_time(timestamp):
    return datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%f")


In [21]:
#Views a single image
import cv2

# Define the paths to the annotation and image files
annotation_file = 'C:/Users/calvi/Desktop/School/ANSC480/DataCode/dataset/labels/train/000451.txt'  # YOLO annotation file
image_file = 'C:/Users/calvi/Desktop/School/ANSC480/DataCode/dataset/images/train/000451.jpg'  # Corresponding image file

# Load the image using OpenCV
image = cv2.imread(image_file)

# Get the image dimensions
img_height, img_width, _ = image.shape

# Read the annotation file
with open(annotation_file, 'r') as f:
    annotations = f.readlines()

# Loop through each annotation and draw the corresponding bounding box
for annotation in annotations:
    # Split the annotation into components
    parts = annotation.strip().split()
    class_id = int(parts[0])  # Class ID
    x_center = float(parts[1])  # Normalized x-center
    y_center = float(parts[2])  # Normalized y-center
    width = float(parts[3])  # Normalized width
    height = float(parts[4])  # Normalized height
    
    # Convert normalized coordinates to pixel coordinates
    x_center = int(x_center * img_width)
    y_center = int(y_center * img_height)
    width = int(width * img_width)
    height = int(height * img_height)

    # Calculate the top-left and bottom-right corners of the bounding box
    x1 = x_center - width // 2
    y1 = y_center - height // 2
    x2 = x_center + width // 2
    y2 = y_center + height // 2

    # Draw the bounding box on the image
    color = (0, 255, 0)  # Green color for the bounding box
    thickness = 2  # Line thickness
    image = cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)
    # Optionally, put the class ID text near the bounding box
    font = cv2.FONT_HERSHEY_SIMPLEX
    image = cv2.putText(image, str(class_id), (x1, y1 - 10), font, 0.5, (0, 255, 0), 2)

# Display the image with bounding boxes
cv2.imshow('Image with Bounding Boxes', image)

# Wait for a key press and close the image window
cv2.waitKey(0)
cv2.destroyAllWindows()


In [57]:
#Views a single image, annotation from the annotated dataset
from PIL import Image, ImageDraw
image_file = 'C:/Users/calvi/Desktop/School/ANSC480/DataCode/dataset/images/train/000025.jpg'
# Load the annotations
with open("C:/Users/calvi/Desktop/School/ANSC480/DataCode/annotated/2019_11_28/000113/output.json", "r") as f:
    annotations = json.load(f)

# Select the frame numbers you want to inspect manually
frames_to_check = [5, 10, 15]  # Example frame numbers

# Define the image extension (e.g., .jpg)
image_extension = ".jpg"
image = Image.open(image_file)
# Function to display image with bounding boxes
def display_image_with_bbox(image_path, bbox):
    
    draw = ImageDraw.Draw(image)

    x, y, w, h = bbox["x"], bbox["y"], bbox["width"], bbox["height"]
    draw.rectangle([x, y, x + w, y + h], outline="red", width=2)

      # This will open the image in the default viewer

# Loop through the selected frames
  
for pig in annotations["objects"]:
    last_fnum_bbox = 0
    for frame in pig["frames"]:
        fnum = frame["frameNumber"]
        if fnum > 25:
            break
        else:
            last_fnum_bbox = frame["bbox"]        
    display_image_with_bbox(image_path, last_fnum_bbox)
image.show()

In [63]:
print(frame_annotations[25])
framelists["objects"]

['4 0.426172 0.425694 0.210156 0.215278', '5 0.635938 0.718056 0.140625 0.255556', '0 0.865234 0.499306 0.111719 0.234722', '1 0.640625 0.332639 0.150000 0.154167', '2 0.497656 0.258333 0.193750 0.188889', '3 0.261328 0.792361 0.252344 0.404167', '6 0.229687 0.588889 0.182812 0.294444', '7 0.648828 0.684722 0.167969 0.291667']


In [ ]:
#Creates folder with bounding boxes on the images, currently set to just pigs eating at feeder, each new instance gets its own frame
import os
import cv2
import numpy as np

# Directory to save images with bounding boxes
output_dir = "bounded_pigs_np"
os.makedirs(output_dir, exist_ok=True)

# Loop through videos and npy files
for i in range(0, len(data_november_29)): #len(data_november_29) or 50
    frames = load_frames(200 + i)
    for tracklet in range(0, data_november_29[i].shape[0]):
        if data_november_29[i].size == 0:
            continue
        pig_data = data_november_29[i][tracklet]
        pig_times = load_times(200 + i) #Clips start at 200 for November 29th
        
        for frame_data in pig_data:
            if is_eating(frame_data[13]) and at_feeder(frame_data[12]):
                frame = frames[int(frame_data[0])]
                x1 = int(frame_data[1])
                y1 = int(frame_data[2])
                x2 = int(frame_data[3])
                y2 = int(frame_data[4])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                save_path = os.path.join(output_dir, f"{video_index}_{frame_count:05}.jpg")
                cv2.imwrite(save_path, frame)
                saved_frames += 1
                frame_count += 1
#            cv2.putText(frame, f"Pig {pig_id}", (x1, y1 - 10),
 #                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
    

    print(f"✅ Video {video_index}: Saved {saved_frames} annotated frames")

print("🎉 All videos processed.")


In [65]:
#3
#Crop november 29th pigs based on video and tracklet
import os
import cv2

# Assumes these functions/variables are defined elsewhere:
# - data_november_29: list of np.arrays for each video
# - load_frames(video_index): loads list of frames for the given video index

output_base = "cropped_pigs_by_tracklet"
os.makedirs(output_base, exist_ok=True)

total_crops = 0

for video_index in range(len(data_november_29)):
    frames = load_frames(200 + video_index)  # Clip numbers start at 200

    video_data = data_november_29[video_index]
    if video_data.size == 0:
        continue

    for tracklet_index in range(video_data.shape[0]):
        tracklet = video_data[tracklet_index]

        # Create output directory: cropped_pigs_by_tracklet/{video_index}/{tracklet_index}/
        out_dir = os.path.join(output_base, f"{video_index}", f"{tracklet_index}")
        os.makedirs(out_dir, exist_ok=True)

        frame_data = tracklet[0]
        frame_num = int(frame_data[0])
        x1, y1 = int(frame_data[1]), int(frame_data[2])
        x2, y2 = int(frame_data[3]), int(frame_data[4])

        if frame_num >= len(frames):
            continue

        frame = frames[frame_num]
        crop = frame[y1:y2, x1:x2]

        out_path = os.path.join(out_dir, f"{frame_num:04}.jpg")
        cv2.imwrite(out_path, crop)
        total_crops += 1

    del frames  # Save memory

print(f"✅ Saved {total_crops} cropped pig images by tracklet.")


✅ Saved 1988 cropped pig images by tracklet.


In [8]:
#creates a folder of bounding boxes for "annotated ground truth" data
import json
import cv2
import os

# Replace with your actual JSON file path
json_file = "C:/Users/calvi/Desktop/School/ANSC480/DataCode/annotated/2019_11_05/000009/output.json"
with open(json_file, "r") as f:
    framelists = json.load(f)

# Video path (for extracting frames)
video_path = "C:/Users/calvi/Desktop/School/ANSC480/DataCode/annotated/2019_11_05/000009/color.mp4"
cap = cv2.VideoCapture(video_path)

max_frame_count = 0  # For loading in different days, you'll need to adjust the names of the files to avoid overlap

if not cap.isOpened():
    print("Error: Could not open video.")
else:
    max_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("Total number of frames:", max_frame_count)

cap.release()

# Image dimensions (required for normalization)
IMG_WIDTH = 1280  # adjust to match your dataset
IMG_HEIGHT = 720

# Output directory for images with bounding boxes
output_dir_bounded = "dataset/bounded_pigs"
os.makedirs(output_dir_bounded, exist_ok=True)

def convert_to_yolo_bbox(x, y, w, h):
    x_center = (x + w / 2) / IMG_WIDTH
    y_center = (y + h / 2) / IMG_HEIGHT
    width = w / IMG_WIDTH
    height = h / IMG_HEIGHT
    return x_center, y_center, width, height

# Group annotations by frameNumber
frame_annotations = {}
last_seen = {}

for pig in framelists["objects"]:
    pig_id = int(pig["id"])  # class label
    dict_by_frame = {
        item["frameNumber"]: {k: v for k, v in item.items() if k != "frame_number"}
        for item in pig["frames"]
        }
    for fnum in range(0, max_frame_count):
        frame_id = fnum // 3
        if frame_id in dict_by_frame.keys():
            bbox = dict_by_frame[frame_id]["bbox"]
            x, y, w, h = bbox["x"], bbox["y"], bbox["width"], bbox["height"]
            x_c, y_c, bw, bh = convert_to_yolo_bbox(x, y, w, h)
            label = f"{pig_id} {x_c:.6f} {y_c:.6f} {bw:.6f} {bh:.6f}"
            frame_annotations.setdefault(fnum, []).append(label)
            last_seen[pig_id] = label
        else:
            frame_annotations.setdefault(fnum, []).append(last_seen[pig_id])

# Iterate through frames and draw bounding boxes on images
cap = cv2.VideoCapture(video_path)
frame_idx = 0
bounded_image_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Check if this frame has annotations
    if frame_idx in frame_annotations:
        # Get annotations for the current frame
        for annotation in frame_annotations[frame_idx]:
            parts = annotation.split()
            pig_id = int(parts[0])
            # Convert YOLO bounding box to pixel coordinates
            x_center, y_center, width, height = map(float, parts[1:])
            x = int((x_center - width / 2) * IMG_WIDTH)
            y = int((y_center - height / 2) * IMG_HEIGHT)
            w = int(width * IMG_WIDTH)
            h = int(height * IMG_HEIGHT)

            # Draw the bounding box and label on the image
            color = (0, 255, 0)  # Green color for bounding box
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

            # Add the pig ID text
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, f"Pig {pig_id}", (x, y - 10), font, 0.9, color, 2)

        # Save the modified image
        output_image_path = os.path.join(output_dir_bounded, f"frame_{frame_idx+fnum_offset:06}.jpg")
        cv2.imwrite(output_image_path, frame)
        bounded_image_count += 1

    frame_idx += 1

cap.release()
print(f"✅ Processed and saved {bounded_image_count} frames with bounding boxes.")


Total number of frames: 1800
✅ Processed and saved 1800 frames with bounding boxes.


In [4]:
#4
#Extract frame annotation data for YOLO training
import json
import os

# Replace with your actual JSON file path
#json_file = "C:/Users/calvi/Desktop/School/ANSC480/DataCode/annotated/2019_11_28/000113/output.json"
json_file = "C:/Users/calvi/Desktop/School/ANSC480/DataCode/annotated/2019_12_02/000005/output.json"
with open(json_file, "r") as f:
    framelists = json.load(f)

#video_path = "C:/Users/calvi/Desktop/School/ANSC480/DataCode/annotated/2019_11_28/000113/color.mp4"  # Replace with your actual path
video_path = "C:/Users/calvi/Desktop/School/ANSC480/DataCode/annotated/2019_12_02/000005/color.mp4"
cap = cv2.VideoCapture(video_path)

max_frame_count = 0
fnum_offset = 0 #For loading in different days, you'll need to adjust the names of the files to avoid overlap

if not cap.isOpened():
    print("Error: Could not open video.")
else:
    max_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("Total number of frames:", max_frame_count)

cap.release()

# Image dimensions (required for normalization)
IMG_WIDTH = 1280  # adjust to match your dataset
IMG_HEIGHT = 720

# Output directory for YOLO label files
output_dir_train = "dataset/labels/train"
output_dir_test = "dataset/labels/test"
output_dir_val = "dataset/labels/val"
#os.makedirs(output_dir, exist_ok=True)
os.makedirs(output_dir_train, exist_ok=True)
os.makedirs(output_dir_test, exist_ok=True)
os.makedirs(output_dir_val, exist_ok=True)

def convert_to_yolo_bbox(x, y, w, h):
    x_center = (x + w / 2) / IMG_WIDTH
    y_center = (y + h / 2) / IMG_HEIGHT
    width = w / IMG_WIDTH
    height = h / IMG_HEIGHT
    return x_center, y_center, width, height

# Group annotations by frameNumber
frame_annotations = {}
last_seen = {}

for pig in framelists["objects"]:
    pig_id = int(pig["id"])  # class label
    dict_by_frame = {
        item["frameNumber"]: {k: v for k, v in item.items() if k != "frame_number"}
        for item in pig["frames"]
        }
    for fnum in range(0, max_frame_count):
        
        #if not frame.get("visible", False):
        #    continue
        #fnum = frame["frameNumber"]
        frame_id = fnum // 3 #They measured every third frame
        if frame_id in dict_by_frame.keys():
            bbox = dict_by_frame[frame_id]["bbox"]
            x, y, w, h = bbox["x"], bbox["y"], bbox["width"], bbox["height"]
            x_c, y_c, bw, bh = convert_to_yolo_bbox(x, y, w, h)
            label = f"{pig_id} {x_c:.6f} {y_c:.6f} {bw:.6f} {bh:.6f}"
            frame_annotations.setdefault(fnum, []).append(label)
            
            last_seen[pig_id] = label
        else:
            frame_annotations.setdefault(fnum, []).append(last_seen[pig_id])

#all_frame_nums = sorted(frame_annotations.keys())
#for fnum in all_frame_nums:
#    present_ids = {int(lbl.split()[0]) for lbl in frame_annotations[fnum]}
#    for pig_id in range(8):  # Ensure pigs 0–7
#        if pig_id not in present_ids and pig_id in last_seen:
            # Use the last seen bbox if pig missing
#            frame_annotations[fnum].append(last_seen[pig_id])

# Save .txt file for each frame
for fnum, labels in frame_annotations.items():
    label_path = os.path.join(output_dir_train, f"{fnum+fnum_offset:06}.txt")
    if fnum > len(frame_annotations.items()) * 0.9:
        label_path = os.path.join(output_dir_val, f"{fnum+fnum_offset:06}.txt")
    elif fnum > len(frame_annotations.items()) * 0.7:
        label_path = os.path.join(output_dir_test, f"{fnum+fnum_offset:06}.txt")
    with open(label_path, "w") as f:
        f.write("\n".join(labels))

print("✅ Labels converted and saved.")


Total number of frames: 1800
✅ Labels converted and saved.


In [15]:
len(framelists["objects"][0]["frames"])

2

In [122]:
#5
#creates cropped images from annotated data
import os
import cv2
import json

# Input video and annotation
paths = ["2019_11_05/000002", "2019_11_05/000009", "2019_11_11/000016","2019_11_11/000028","2019_11_11/000036", "2019_11_15/000033",
        "2019_11_22/000010","2019_11_28/000113","2019_12_02/000005","2019_12_02/000208",]
iter_offset = 0
for path in paths:
    video_path = f"C:/Users/calvi/Desktop/School/ANSC480/DataCode/annotated/{path}/color.mp4"
    json_file = f"C:/Users/calvi/Desktop/School/ANSC480/DataCode/annotated/{path}/output.json"
    
    with open(json_file, "r") as f:
        framelists = json.load(f)
    
    # Output directory for cropped images by class
    base_output_dir = "cropped_pigs"
    os.makedirs(base_output_dir, exist_ok=True)
    for i in range(8):
        os.makedirs(os.path.join(base_output_dir, str(i)), exist_ok=True)
    
    # Open video
    cap = cv2.VideoCapture(video_path)
    max_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Parse JSON annotations
    frame_data_by_id = {}
    last_seen = {}
    for pig in framelists["objects"]:
        pig_id = int(pig["id"])  # class label
        dict_by_frame = {
            item["frameNumber"]: {k: v for k, v in item.items() if k != "frame_number"}
            for item in pig["frames"]
            }
        for fnum in range(0, max_frame_count, 30):
            
            #if not frame.get("visible", False):
            #    continue
            #fnum = frame["frameNumber"]
            frame_id = fnum // 3 #They measured every third frame
            if frame_id in dict_by_frame.keys():
                bbox = dict_by_frame[frame_id]["bbox"]
                frame_data_by_id.setdefault(fnum, []).append((pig_id, bbox))
                
                last_seen[pig_id] = bbox
            else:
                frame_data_by_id.setdefault(fnum, []).append((pig_id, last_seen[pig_id]))
    
    # Extract bounding boxes as images
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    frame_idx = 0
    count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or frame_idx >= max_frame_count:
            break
    
        if frame_idx in frame_data_by_id:
            for pig_id, bbox in frame_data_by_id[frame_idx]:
                x, y, w, h = int(bbox["x"]), int(bbox["y"]), int(bbox["width"]), int(bbox["height"])
                crop = frame[y:y+h, x:x+w]
                out_path = os.path.join(base_output_dir, str(pig_id), f"{frame_idx+iter_offset:06}_{pig_id}.jpg")
                cv2.imwrite(out_path, crop)
                count += 1
    
        frame_idx += 1
    
    cap.release()
    iter_offset += 1800
    print(f"✅ Saved {count} cropped images.")
    


✅ Saved 480 cropped images.
✅ Saved 480 cropped images.
✅ Saved 480 cropped images.
✅ Saved 480 cropped images.
✅ Saved 480 cropped images.
✅ Saved 480 cropped images.
✅ Saved 480 cropped images.
✅ Saved 480 cropped images.
✅ Saved 480 cropped images.
✅ Saved 480 cropped images.


In [324]:
#Creates a video out of the cropped images
import cv2
import os

# Directory containing cropped pig images
image_folder = 'dataset/cropped_pigs'
output_video_path = 'annotated_pigs_video.mp4'

# Get all image filenames and sort them
images = sorted([img for img in os.listdir(image_folder) if img.endswith(('.png', '.jpg', '.jpeg'))])

# Make sure there are images
if not images:
    raise Exception("No images found in the folder.")

# Read the first image to get dimensions
first_image = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = first_image.shape

# Define video codec and create VideoWriter object
fps = 10  # Frames per second
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Write each image to the video
for image_name in images:
    img_path = os.path.join(image_folder, image_name)
    frame = cv2.imread(img_path)
    video.write(frame)

video.release()
print("✅ Video saved as", output_video_path)


✅ Video saved as annotated_pigs0_video.mp4


In [5]:

#Transfer Frame data for Yolo training
import cv2
import os

# Path to your input video
output_image_dir_train = "dataset/images/train"
output_image_dir_test = "dataset/images/test"
output_image_dir_val = "dataset/images/val"

os.makedirs(output_image_dir_train, exist_ok=True)
os.makedirs(output_image_dir_test, exist_ok=True)
os.makedirs(output_image_dir_val, exist_ok=True)

#os.makedirs(output_image_dir, exist_ok=True)

# Only extract frames that appear in annotation file
target_frames = set(frame_annotations.keys())  # from previous step

# Open the video
cap = cv2.VideoCapture(video_path)

frame_idx = 0
saved = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_idx in target_frames:
        filename = f"{frame_idx+fnum_offset:06}.jpg"
        out_path = os.path.join(output_image_dir_train, filename)
        if frame_idx > len(target_frames) * 0.9:
            out_path = os.path.join(output_image_dir_val, filename)
        elif frame_idx > len(target_frames) * 0.7:
            out_path = os.path.join(output_image_dir_test, filename)
        
        cv2.imwrite(out_path, frame)
        saved += 1

    frame_idx += 1

cap.release()
print(f"✅ Saved {saved} frames")



✅ Saved 1800 frames


dataset/images/val\001799.jpg


In [326]:

#Need to train model to identify each of the 8 pigs at each frame
from ultralytics import YOLO
# Clear CUDA memory
#del model
#torch.cuda.empty_cache()

# Load the YOLO model (choose a pre-trained version, e.g., YOLOv5s)
yolo_model = YOLO("yolo11n.pt")  # you can choose other pre-trained weights like yolov5m.pt, yolov5l.pt, etc.

# Train the model using the data
yolo_model.train(data='C:/Users/calvi/Desktop/School/ANSC480/DataCode/dataset/datasets.yaml', epochs=50, imgsz=640, batch=16)
val_results = yolo_model.val()
#print(results)

New https://pypi.org/project/ultralytics/8.3.127 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.123  Python-3.10.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=C:/Users/calvi/Desktop/School/ANSC480/DataCode/dataset/datasets.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train21, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina

train: Scanning C:\Users\calvi\Desktop\School\ANSC480\DataCode\dataset\labe


val: Fast image access  (ping: 0.30.1 ms, read: 297.370.3 MB/s, size: 280.4 KB)


val: Scanning C:\Users\calvi\Desktop\School\ANSC480\DataCode\dataset\labels


Plotting labels to runs\detect\train21\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000833, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train21
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


50 epochs completed in 0.217 hours.
Optimizer stripped from runs\detect\train21\weights\last.pt, 5.4MB
Optimizer stripped from runs\detect\train21\weights\best.pt, 5.4MB

Validating runs\detect\train21\weights\best.pt...
Ultralytics 8.3.123  Python-3.10.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLO11n summary (fused): 100 layers, 2,583,712 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP


                   all        179       1432      0.497      0.585      0.576      0.492
                 pig_1        179        179      0.995          1      0.995      0.977
                 pig_2        179        179      0.995          1      0.995      0.987
                 pig_3        179        179      0.639          1      0.995      0.953
                 pig_4        179        179          0          0       0.17     0.0798
                 pig_5        179        179     0.0273     0.0447     0.0463     0.0205
                 pig_6        179        179      0.931          1      0.995      0.782
                 pig_7        179        179          0          0    0.00364    0.00262
                 pig_8        179        179      0.384      0.637      0.412      0.134
Speed: 0.5ms preprocess, 1.4ms inference, 0.0ms loss, 3.4ms postprocess per image
Results saved to runs\detect\train21
Ultralytics 8.3.123  Python-3.10.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX

val: Scanning C:\Users\calvi\Desktop\School\ANSC480\DataCode\dataset\labels
                 Class     Images  Instances      Box(P          R      mAP


                   all        179       1432      0.496      0.585      0.576      0.492
                 pig_1        179        179      0.995          1      0.995       0.98
                 pig_2        179        179      0.995          1      0.995      0.986
                 pig_3        179        179       0.64          1      0.995      0.953
                 pig_4        179        179          0          0       0.17     0.0797
                 pig_5        179        179     0.0273     0.0447     0.0465     0.0206
                 pig_6        179        179       0.93          1      0.995      0.782
                 pig_7        179        179          0          0     0.0036     0.0026
                 pig_8        179        179      0.384      0.638      0.412      0.134
Speed: 0.4ms preprocess, 4.0ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to runs\detect\train212


In [ ]:
import matplotlib.pyplot as plt

# Extract key metrics
val_results = results
metrics = val_results.results_dict  # This gives a dictionary of metrics
# Print to see what's inside
print(metrics)
# Separate metrics into keys and values
keys = list(metrics.keys())
values = list(metrics.values())

# Create a bar chart
plt.figure(figsize=(10, 6))
plt.barh(keys, values, color='skyblue')
plt.xlabel("Value")
plt.title("YOLO Validation Metrics")
plt.tight_layout()
plt.show()


In [327]:
frames = load_frames(200+5)
frame = frames[1].copy()
results = yolo_model(frame)
results[0].show()


0: 384x640 2 pig_6s, 39.0ms
Speed: 4.3ms preprocess, 39.0ms inference, 16.6ms postprocess per image at shape (1, 3, 384, 640)


In [123]:
#6
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

# Dataset setup
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
])

train_ds = torchvision.datasets.ImageFolder(root='cropped_pigs', transform=transform)
val_ds = torchvision.datasets.ImageFolder(root='cropped_pigs_val', transform=transform)
#train_size = int(0.8 * len(dataset))
#val_size = len(dataset) - train_size
#train_ds, val_ds = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model
resnet_model = torchvision.models.resnet18(pretrained=True)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 8)  # 8 pigs
resnet_model = resnet_model.to(device)

# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_model.parameters(), lr=0.0003)

for epoch in range(30):
    resnet_model.train()
    total_train_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = resnet_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item() * inputs.size(0)

    avg_train_loss = total_train_loss / len(train_loader.dataset)

    # Validation phase
    resnet_model.eval()
    total_val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = resnet_model(inputs)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = total_val_loss / len(val_loader.dataset)
    val_accuracy = correct / total

    print(f"📘 Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f} | Val Loss = {avg_val_loss:.4f} | Val Acc = {val_accuracy:.4f}")

# Save model
torch.save(resnet_model.state_dict(), "pig_classifier.pt")



📘 Epoch 1: Train Loss = 1.1093 | Val Loss = 3.3422 | Val Acc = 0.2271
📘 Epoch 2: Train Loss = 0.5585 | Val Loss = 2.8761 | Val Acc = 0.2250
📘 Epoch 3: Train Loss = 0.3906 | Val Loss = 3.8595 | Val Acc = 0.2396
📘 Epoch 4: Train Loss = 0.2999 | Val Loss = 3.7979 | Val Acc = 0.2146
📘 Epoch 5: Train Loss = 0.2416 | Val Loss = 3.9367 | Val Acc = 0.1771
📘 Epoch 6: Train Loss = 0.1775 | Val Loss = 4.2340 | Val Acc = 0.2271
📘 Epoch 7: Train Loss = 0.1717 | Val Loss = 4.6859 | Val Acc = 0.2458
📘 Epoch 8: Train Loss = 0.1707 | Val Loss = 4.4801 | Val Acc = 0.1792
📘 Epoch 9: Train Loss = 0.1187 | Val Loss = 4.5436 | Val Acc = 0.1750
📘 Epoch 10: Train Loss = 0.0954 | Val Loss = 5.2423 | Val Acc = 0.2542
📘 Epoch 11: Train Loss = 0.0917 | Val Loss = 4.2470 | Val Acc = 0.1854
📘 Epoch 12: Train Loss = 0.0839 | Val Loss = 4.9440 | Val Acc = 0.2188
📘 Epoch 13: Train Loss = 0.0832 | Val Loss = 5.4216 | Val Acc = 0.2687
📘 Epoch 14: Train Loss = 0.0769 | Val Loss = 6.0388 | Val Acc = 0.2458
📘 Epoch 15: Tra

In [108]:
#Do one sample
# Set the model to evaluation mode
resnet_model.eval()

# Get one sample from the training dataset
img, true_label = val_ds[7]  # Change the index if you want a different sample

# Add batch dimension and send to device
img_tensor = img.unsqueeze(0).to(device)

# Run it through the model
with torch.no_grad():
    output = resnet_model(img_tensor)
    predicted_label = output.argmax(dim=1).item()
print(output)
# Map index to class name
class_names = dataset.classes  # From ImageFolder

print(f"✅ True label: {class_names[true_label]}")
print(f"🔮 Predicted label: {class_names[predicted_label]}")
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F

plt.imshow(F.to_pil_image(img))
plt.title(f"True: {class_names[true_label]}, Predicted: {class_names[predicted_label]}")
plt.axis('off')
plt.show()


tensor([[ 6.3978,  0.8172, -3.7773, -3.1269,  1.7239, -2.3212, -1.9694,  1.1144]], device='cuda:0')
✅ True label: 0
🔮 Predicted label: 0


<Figure size 640x480 with 1 Axes>

In [126]:
#Detailed prevision and recall
from sklearn.metrics import accuracy_score, precision_score, classification_report
dataset = torchvision.datasets.ImageFolder(root='cropped_pigs_val', transform=transform)
resnet_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = resnet_model(inputs)
        preds = outputs.argmax(dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Accuracy and Precision (macro = unweighted mean across classes)
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='macro')  # or 'weighted'

print(f"\n📊 Validation Accuracy: {accuracy:.4f}")
print(f"🎯 Validation Precision (macro): {precision:.4f}")

# Optional: Show precision/recall/f1 for each pig class
print("\n🔍 Detailed classification report:")
print(classification_report(all_labels, all_preds, target_names=dataset.classes))



📊 Validation Accuracy: 0.1854
🎯 Validation Precision (macro): 0.2017

🔍 Detailed classification report:
              precision    recall  f1-score   support

           0       0.35      0.68      0.46        60
           1       0.33      0.03      0.06        60
           2       0.00      0.00      0.00        60
           3       0.29      0.03      0.06        60
           4       0.19      0.43      0.27        60
           5       0.00      0.00      0.00        60
           6       0.42      0.28      0.34        60
           7       0.03      0.02      0.02        60

    accuracy                           0.19       480
   macro avg       0.20      0.19      0.15       480
weighted avg       0.20      0.19      0.15       480



In [131]:
#7
import math
#imperfect function for estimating what pig is currently the focus of the current frame data
def yolo_which_pig(frame, frame_data, prediction):
    center_x = (frame_data[3] - frame_data[1]) / 2
    center_y = (frame_data[4] - frame_data[2]) / 2
    best_center_dist = math.inf
    best_center_class = -1
    for box in prediction.boxes:
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        conf = box.conf[0].item()
        cls = int(box.cls[0].item())
        pred_center_x = (x2-x1)/2
        pred_center_y = (y2-y1)/2
        if conf > 0.8:
            if math.dist((pred_center_x, pred_center_y), (center_x, center_y)) < best_center_dist and math.dist((pred_center_x, pred_center_y), (center_x, center_y)) < 200:
                best_center_dist = math.dist((pred_center_x, pred_center_y), (center_x, center_y)) 
                best_center_class = cls
    return best_center_class

resnet_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def resnet_which_pig(frame, chosen_model):
    device = next(chosen_model.parameters()).device
    frame = frame.to(device)
    #frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #x1, y1, x2, y2 = int(frame_data[1]), int(frame_data[2]), int(frame_data[3]), int(frame_data[4])
    
    #crop = frame_rgb[y1:y2, x1:x2]  # Ensure y coordinates come first in slicing

    # Transform and add batch dimension
    #input_tensor = resnet_transform(crop).unsqueeze(0).to(device)

    # Get prediction
    predicted_class = 0
    with torch.no_grad():
        output = chosen_model(frame)
        #print(output)
        predicted_class = output.argmax(dim=1).item()

    #print(f"Predicted class: {predicted_class}")
    #cv2.imshow('Image with Bounding Box', crop)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    return predicted_class


In [133]:
#8
pigs_feeding_times = [[]] * 8 
#pig0_feeding_times = []
for i in range(0, len(data_november_29)): #len(data_november_29) or 50
    frames = load_frames(200 + i)
    if not os.path.exists(f'cropped_pigs_by_tracklet/{i}'): continue
    tracklet_dataset = torchvision.datasets.ImageFolder(root=f'cropped_pigs_by_tracklet/{i}', transform=transform)
    for tracklet in range(0, data_november_29[i].shape[0]):
        if data_november_29[i].size == 0:
            continue
            
        img, true_label = tracklet_dataset[tracklet]
        img_tensor = img.unsqueeze(0).to(device)
        #print(true_label)
        
        pig_data = data_november_29[i][tracklet]
        pig_times = load_times(200 + i) #Clips start at 200 for November 29th
        
        
        pig_id = resnet_which_pig(img_tensor, resnet_model)

        #frame_data = pig_data[0]
        #x1, y1, x2, y2 = int(frame_data[1]), int(frame_data[2]), int(frame_data[3]), int(frame_data[4])
        #crop = frames[0][y1:y2, x1:x2]
        #cv2.imshow('Image with Bounding Box', crop)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()


        for frame_data in pig_data:
            if is_eating(frame_data[13]) and at_feeder(frame_data[12]):
                
                if pig_id == -1:
                    pigs_feeding_times[tracklet % 8].append(pig_times[int(frame_data[0])])
                else:
                    pigs_feeding_times[pig_id].append(pig_times[int(frame_data[0])])
                #pig0_feeding_times.append(pig0_times[int(frame_data[0])])
                
                #display_frame(frame_data, frames[int(frame_data[0])])
                #pigs_feeding_times[which_pig(frame, frame_data, model)] = (pig0_times[int(frame_data[0])], duration)
                #break #Commenting this out will probably result in too much memory being used, recording every single frame that has feeding behavior
                
                #break
    del frames
pigs_feeding_times = [sorted(times) for times in pigs_feeding_times] #All pigs feeding times, only one instance max per 5 minute video

#for pig in range(0, len(pigs)):
#    pigs_feeding_times[pig] = sorted(pigs_feeding_times[pig], key=lambda x: x[0])

In [ ]:
model.eval()

#for i in range(1, 10):
#    with torch.no_grad():
        #X_tensor = torch.tensor(intervals[-(SEQ_LENGTH+i):-i]).unsqueeze(0).unsqueeze(-1)
        #test_input = torch.tensor(intervals[-(SEQ_LENGTH+i):-i]).unsqueeze(0).unsqueeze(-1)
predicted = model(X_test_tensor)
        #actual = intervals[i] - intervals[i-1]
print(predicted)
print(y_test_tensor)

predicted = model(X_train_tensor)
print(predicted)
print(y_train_tensor)
        #print(f"/n⏱️ Predicted next feeding interval: {predicted:.2f} minutes")
        #print(f"/n⏱️ Actual next feeding interval: {actual:.2f} minutes")

In [134]:
#9
def condense_feeding(feeding_times):
    feeding_durations = []
    start_time = 0
    prev_time = 0
    for time in feeding_times:
        if time - prev_time < 0.15: #9 seconds
            prev_time = time
        else:
            feeding_durations.append((start_time, prev_time - start_time))
            #feeding_durations = np.append(feeding_durations, (start_time, prev_time - start_time))
            start_time = time
            prev_time = time
    feeding_durations.append((start_time, prev_time - start_time))
    #feeding_durations = np.append(feeding_durations, (start_time, prev_time - start_time))
    return feeding_durations
            

In [ ]:
#len(feeding_times)
#len(feeding_times[0])
#print(feeding_times)
#print(pigs_feeding_times)
len(durations)
durations[:100]

In [151]:
#Flatten pigs_feeding_times data
pigs_feeding_times_flat = []
for i in pigs_feeding_times:
    for j in i:
        pigs_feeding_times_flat.append(j)

In [320]:
#10
#Pretty much the RFID tag feeding prediction

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from datetime import datetime


feeding_times = pigs_feeding_times[0]  # Choose pig 0 or any specific pig

#feeding_times = sorted(pigs_feeding_times_flat) #To just get all feeding times for all pigs
intervals = np.array([t.timestamp() - feeding_times[0].timestamp() for t in feeding_times], dtype=np.float32) / 60


durations = condense_feeding(intervals)
#durations = rfid_tags_durations #Use fake RFID tag data instead (run at bottom first)

import numpy as np

def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x_seq = data[i:i+seq_length]  # List of tuples
        last_start_time = x_seq[-1][0]

        # Subtract last start time from all start times in the sequence
        adjusted_seq = [(last_start_time - start_time, duration) for (start_time, duration) in x_seq]
        xs.append(adjusted_seq)

        # Compute the target delta between next and last start time
        next_start_time = data[i + seq_length][0]
        delta = next_start_time - last_start_time
        ys.append(delta)

    return np.array(xs, dtype=np.float32), np.array(ys, dtype=np.float32)


SEQ_LENGTH = 5
X, y = create_sequences(durations, SEQ_LENGTH)

permutation = np.random.permutation(len(X))
X = X[permutation]
y = y[permutation]

split_idx = int(0.8 * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]


#X_train_tensor = torch.tensor(X_train).unsqueeze(-1)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)

y_train_tensor = torch.tensor(y_train).unsqueeze(-1)
#X_test_tensor = torch.tensor(X_test).unsqueeze(-1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

y_test_tensor = torch.tensor(y_test).unsqueeze(-1)
#X_tensor = torch.tensor(X).unsqueeze(-1)  # shape: (batch, seq, 1)
#y_tensor = torch.tensor(y).unsqueeze(-1)


class FeedLSTM(nn.Module):
    def __init__(self, input_size=2, hidden_size=64):
        super(FeedLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]           
        out = self.relu(self.fc1(out))  
        out = self.fc2(out)     
        return out

model = FeedLSTM()
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)


EPOCHS = 100
TestLoss_tolerance = 0.05
for epoch in range(EPOCHS):
    model.train()
    output = model(X_train_tensor)
    loss = loss_fn(output, y_train_tensor)
    #print(output, y_train_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 20 == 0 or epoch < 20:
        model.eval()
        with torch.no_grad():
            test_output = model(X_test_tensor)
            test_loss = loss_fn(test_output, y_test_tensor)
        print(f"Epoch {epoch}: Train Loss = {loss.item():.4f}, Test Loss = {test_loss.item():.4f}")


model.eval()
with torch.no_grad():
    recent_sequence = np.array(durations[-SEQ_LENGTH:], dtype=np.float32)
    test_input = torch.tensor([recent_sequence])  # shape: (1, SEQ_LENGTH, 2)
    predicted_start_time = model(test_input).item()
    actual_start_time = durations[-1][0] - durations[-2][0]  # last known start time

    print(f"\n⏱️ Predicted next feeding start time: {predicted_start_time:.2f} minutes since start")
    print(f"⏱️ Actual next feeding time: {actual_start_time:.2f} minutes since start")


    


Epoch 0: Train Loss = 78.0942, Test Loss = 11.5802
Epoch 1: Train Loss = 76.9874, Test Loss = 10.8771
Epoch 2: Train Loss = 76.0050, Test Loss = 10.1505
Epoch 3: Train Loss = 74.9693, Test Loss = 9.2610
Epoch 4: Train Loss = 73.6796, Test Loss = 8.1723
Epoch 5: Train Loss = 72.0301, Test Loss = 6.8719
Epoch 6: Train Loss = 69.9399, Test Loss = 5.5041
Epoch 7: Train Loss = 67.4884, Test Loss = 4.4341
Epoch 8: Train Loss = 65.0064, Test Loss = 4.3040
Epoch 9: Train Loss = 63.1286, Test Loss = 5.9083
Epoch 10: Train Loss = 62.7324, Test Loss = 8.8228
Epoch 11: Train Loss = 63.8470, Test Loss = 10.4763
Epoch 12: Train Loss = 64.5826, Test Loss = 10.0984
Epoch 13: Train Loss = 64.1135, Test Loss = 8.5449
Epoch 14: Train Loss = 63.0014, Test Loss = 6.7537
Epoch 15: Train Loss = 61.9330, Test Loss = 5.2930
Epoch 16: Train Loss = 61.2901, Test Loss = 4.3558
Epoch 17: Train Loss = 61.1078, Test Loss = 3.8641
Epoch 18: Train Loss = 61.1987, Test Loss = 3.6447
Epoch 19: Train Loss = 61.3221, Test

In [321]:
#Just display some of the test results
for i in range(0, len(X_test_tensor)):
    input_tensor = X_test_tensor[i].unsqueeze(0)  # Shape: [1, N]
    model.eval()
    with torch.no_grad():
        test_output = model(input_tensor)
        test_loss = loss_fn(test_output, y_test_tensor[0])
    #print(f"Epoch {epoch}: Train Loss = {loss.item():.4f}, Test Loss = {test_loss.item():.4f}")
    print("Predicted:" + str(round(test_output.item(), 2)) + " Actual:" + str(round(y_test_tensor[i].item(), 2)))

Predicted:4.26 Actual:6.81
Predicted:1.52 Actual:0.33
Predicted:1.58 Actual:0.57
Predicted:1.07 Actual:4.83
Predicted:1.21 Actual:6.76
Predicted:1.0 Actual:1.08
Predicted:1.45 Actual:1.66
Predicted:0.58 Actual:0.3
Predicted:1.15 Actual:2.17
Predicted:1.1 Actual:0.72
Predicted:1.28 Actual:1.54
Predicted:2.09 Actual:3.97
Predicted:1.79 Actual:0.47
Predicted:1.47 Actual:2.63
Predicted:1.05 Actual:0.42
Predicted:1.28 Actual:0.16
Predicted:7.13 Actual:3.63
Predicted:1.47 Actual:5.14
Predicted:1.11 Actual:1.4
Predicted:9.65 Actual:1.85
Predicted:9.24 Actual:2.08
Predicted:7.27 Actual:5.21
Predicted:1.03 Actual:0.59
Predicted:1.33 Actual:2.34
Predicted:3.49 Actual:1.15
Predicted:2.55 Actual:1.65
Predicted:2.01 Actual:0.46
Predicted:1.46 Actual:2.04
Predicted:2.4 Actual:6.46
Predicted:1.21 Actual:2.49
Predicted:1.76 Actual:1.8
Predicted:16.26 Actual:6.86
Predicted:0.99 Actual:0.61
Predicted:0.96 Actual:0.96
Predicted:3.08 Actual:2.22
Predicted:1.08 Actual:0.6
Predicted:3.16 Actual:6.81
Predict

In [294]:
#Generate fake RFID tag data
import random
import math

rfid_tags_durations = [(0, 0.3)]
for i in range(1, 1000):
    # Start with last feeding time
    next_time = rfid_tags_durations[-1][0]
    
    # Add decayed influence from recent durations
    for j in range(min(len(rfid_tags_durations), 5)):
        past_time, past_duration = rfid_tags_durations[-j - 1]
        decay_factor = 1 / (j + 1)  # more recent = higher weight
        next_time += past_duration * decay_factor

    # Random duration
    duration = random.uniform(2, 10)

    rfid_tags_durations.append((next_time, duration))

# Show first few entries
for t, d in rfid_tags_durations[:30]:
    print(f"⏰ {t:.2f}-time — Duration: {d:.2f}minutes")



⏰ 0.00-time — Duration: 0.30minutes
⏰ 0.30-time — Duration: 8.42minutes
⏰ 8.87-time — Duration: 9.89minutes
⏰ 23.07-time — Duration: 2.27minutes
⏰ 33.17-time — Duration: 4.25minutes
⏰ 44.02-time — Duration: 2.34minutes
⏰ 53.39-time — Duration: 8.20minutes
⏰ 66.72-time — Duration: 4.89minutes
⏰ 78.00-time — Duration: 4.21minutes
⏰ 88.83-time — Duration: 9.26minutes
⏰ 104.34-time — Duration: 9.22minutes
⏰ 122.46-time — Duration: 2.98minutes
⏰ 135.17-time — Duration: 4.33minutes
⏰ 147.22-time — Duration: 2.86minutes
⏰ 157.39-time — Duration: 8.39minutes
⏰ 171.24-time — Duration: 2.51minutes
⏰ 180.57-time — Duration: 6.24minutes
⏰ 192.44-time — Duration: 6.67minutes
⏰ 205.73-time — Duration: 6.43minutes
⏰ 219.88-time — Duration: 5.76minutes
⏰ 233.14-time — Duration: 2.11minutes
⏰ 243.18-time — Duration: 8.82minutes
⏰ 257.92-time — Duration: 9.13minutes
⏰ 274.88-time — Duration: 9.04minutes
⏰ 293.11-time — Duration: 3.17minutes
⏰ 306.47-time — Duration: 4.76minutes
⏰ 319.87-time — Duration:

In [328]:
#print(rfid_tags_durations)
for i in rfid_tags_durations:
    print(i)

(0, 0.3)
(0.3, 8.42454766858923)
(8.874547668589232, 9.885469130103509)
(23.072290632987354, 2.272228129176283)
(33.170435883411805, 4.248011887009296)
(44.01585512885773, 2.3407795930747497)
(53.394326858072944, 8.19801854477652)
(66.72389002003804, 4.885154864656367)
(78.0047626190285, 4.211667460583982)
(88.82647763587005, 9.262607067144419)
(104.340963943001, 9.221796084054521)
(122.45884513927513, 2.980075693888324)
(135.1673024023161, 4.3258881044370705)
(147.2151456406184, 2.8621519857098257)
(157.39157067761872, 8.385872239003131)
(171.23985975123878, 2.5054136106978095)
(180.56974730822836, 6.2372660111645715)
(192.43572648839108, 6.673401526691266)
(205.73179734779, 6.429733670322423)
(219.88084830232137, 5.757587837690695)
(233.13816937566773, 2.108340056103577)
(243.18435149129652, 8.818512119050306)
(257.9183433072144, 9.125786145349515)
(274.88150922427747, 9.044103165567211)
(293.10661208376683, 3.170118455218299)
(306.4670068778686, 4.757883877723007)
(319.8697999985372